<a href="https://colab.research.google.com/github/JavZon/Machine_Learning/blob/main/MAE_RMSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

Datasetni chaqirib olamiz

In [3]:
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)

Datasetni 2 qismga bo'lamiz

In [4]:
from sklearn.model_selection import train_test_split
train_set, test_set=train_test_split(df,test_size=0.2, random_state=42)

Datasetni **X_train** ga yuklab olib **median_house_value** ni o'chirib yuboramiz

In [11]:
X_train = train_set.drop(columns='median_house_value')

y = train_set["median_house_value"].copy() # faqat labellarini saqlab oldik

X_num = X_train.drop(columns='ocean_proximity')

**Transformer** yasaymiz. Bu transformer har bir uydagi xonalar soni (rooms_per_household) va har bir uydagi odamlar sonini( population_per_household) aniqlab beradi

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
      pass
    def fit(self, X, y=None):
      return self
    def transform(self, X):
      rooms_per_household=X[:, rooms_ix] / X[:, households_ix]
      population_per_household=X[:, population_ix] / X[:, households_ix]
      return np.c_[X, rooms_per_household, population_per_household]

Sonli ustunlar NaN larning o'rnini to'ldirish uchun **SimpleImputer** va datasetni -1 va 1 oralig'iga keltirish uchun **StandardScaler**ni chaqiramiz

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numeric_pipeline=Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('custom_obj_adder', CustomTransformer()),
    ('std_scaler', StandardScaler())
])

prepared_num=numeric_pipeline.fit_transform(X_num)
prepared_num[0,:]

array([ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
        0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609])

**ColumnTransformer** nomli yangi classni chaqirib olamiz sklearn.compose ning ichidan.Bu class yakuniy full pipeline yaratish uchun ishlatiladi

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_cols=list(X_num.columns)
string_cols=['ocean_proximity']

full_pipeline=ColumnTransformer([
    ('numeric', numeric_pipeline, num_cols),
    ('string_cat',OneHotEncoder(handle_unknown='ignore'), string_cols)
])

In [17]:
X_prepared=full_pipeline.fit_transform(X_train)

X_prepared[:3,:]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ]])

In [18]:
X_train.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,NEAR OCEAN


# **Machine Learning** qismini boshlaymiz.

In [19]:
X_prepared[0,:]

array([ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
        0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ])

**Linear Regression** algorithmini chaqirib olamiz

In [15]:
from sklearn.linear_model import LinearRegression # aslida algorithmlar ko'p hammasi bn keyinroq tanishamiz

LR_model= LinearRegression() # bu classdan object yaratib oldik va LR_model deb nom berdik

**LR_model.fit()** metodi orqali X_prepared tayyor datasetga ishlov beramiz

In [20]:
LR_model.fit(X_prepared,y) # X_prepared - biz tayyorlagan dataset va y - asl label

LinearRegression()

Modelni sinab ko'rish uchun X_traindan 10 ta sample olamiz va shu sample larga mos bo'lgan asl labelning valuelarini ham olamiz, yakunda solishtirish uchun

In [21]:
test_data=X_train.sample(10)
test_label=y.loc[test_data.index]  # y - asl label
test_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
13878,-117.35,34.44,9.0,11810.0,2181.0,6716.0,2081.0,3.1821,INLAND
7804,-118.08,33.89,41.0,834.0,166.0,603.0,179.0,3.7321,<1H OCEAN
17740,-121.76,37.28,17.0,660.0,129.0,431.0,123.0,4.9097,<1H OCEAN
1223,-120.67,38.19,17.0,2967.0,611.0,1387.0,564.0,2.0417,INLAND
16008,-122.44,37.73,43.0,3700.0,684.0,1488.0,623.0,5.5622,NEAR BAY
3079,-119.01,35.44,20.0,3458.0,651.0,1465.0,621.0,2.5806,INLAND
6343,-117.75,34.06,52.0,62.0,9.0,44.0,16.0,0.4999,INLAND
8528,-118.34,33.89,36.0,2274.0,411.0,1232.0,423.0,5.3730,<1H OCEAN
4576,-118.28,34.07,24.0,3247.0,1281.0,2642.0,1182.0,2.4632,<1H OCEAN
17308,-119.59,34.43,28.0,2718.0,542.0,1066.0,442.0,4.2059,<1H OCEAN


In [22]:
test_label

13878     95600.0
7804     167500.0
17740    241000.0
1223      92600.0
16008    313600.0
3079      82500.0
6343     112500.0
8528     244500.0
4576     216700.0
17308    500001.0
Name: median_house_value, dtype: float64

Bu 10 ta tanlab olingan sampleni ham **full_pipeline**dan o'tkazib olib tayyor holga keltirib olamiz

In [23]:
test_prepared=full_pipeline.transform(test_data)
test_prepared

array([[ 1.11303274e+00, -5.63113774e-01, -1.55595157e+00,
         4.21598261e+00,  3.92010757e+00,  4.65210632e+00,
         4.15011525e+00, -3.66894735e-01,  1.00498846e-01,
         1.12566027e-02,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.49050580e-01, -8.20532031e-01,  9.83304186e-01,
        -8.31426778e-01, -8.89025829e-01, -7.24219093e-01,
        -8.42581711e-01, -7.80650284e-02, -3.25060237e-01,
         2.34707767e-02,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.08581839e+00,  7.66100496e-01, -9.21137632e-01,
        -9.11442196e-01, -9.77332497e-01, -8.75491459e-01,
        -9.89580150e-01,  5.40345631e-01, -2.90626595e-02,
         3.51606504e-02,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-5.42338176e-01,  1.19201070e+00, -9.21137632e-01,
         1.49451886e-01,  1.73040852e-01, -3.46985420

Tayyor bo'lgan datasetni **LR_modelning** ichidagi **predict()** metodiga uzatamiz va modelimiz shu 10 ta sample uchun **median_house_value** ni predict qiladi

In [24]:
predicted_labels=LR_model.predict(test_prepared)

In [25]:
predicted_labels

array([ 90847.50130697, 219980.35445123, 253107.09719346,  68499.39824351,
       332499.67149682, 121518.45364971,  53563.06998206, 291306.87120635,
       224348.60513825, 274956.55211046])

Endi chiqqan natijani **asl label** bilan solishtirish uchun dataframe tuzamiz va farqini ko'rishimiz mumkin bo'ladi

In [26]:
df2=pd.DataFrame({'Predicted_labels':predicted_labels, 'Real_labels':test_label})

In [27]:
df2

,Predicted_labels,Real_labels
13878,90847.501307,95600.0
7804,219980.354451,167500.0
17740,253107.097193,241000.0
1223,68499.398244,92600.0
16008,332499.671497,313600.0
3079,121518.453650,82500.0
6343,53563.069982,112500.0
8528,291306.871206,244500.0
4576,224348.605138,216700.0
17308,274956.552110,500001.0


Modelning aniqligini baholash **MAE** (Mean Absolute Error) va **RSME** (Root Mean Squared Error)

In [28]:
# Bu safar biz modelimiz hali ko'rmagan test_set bilan ishlaymiz
test_set

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,47700.0,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,45800.0,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,500001.0,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,218600.0,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,278000.0,NEAR OCEAN
...,...,...,...,...,...,...,...,...,...,...
15362,-117.22,33.36,16.0,3165.0,482.0,1351.0,452.0,4.6050,263300.0,<1H OCEAN
16623,-120.83,35.36,28.0,4323.0,886.0,1650.0,705.0,2.7266,266800.0,NEAR OCEAN
18086,-122.05,37.31,25.0,4111.0,538.0,1585.0,568.0,9.2298,500001.0,<1H OCEAN
2144,-119.76,36.77,36.0,2507.0,466.0,1227.0,474.0,2.7850,72300.0,INLAND


In [29]:
X_test=test_set.drop("median_house_value", axis=1)

Y_label=test_set['median_house_value'].copy()

In [30]:
Y_label

20046     47700.0
3024      45800.0
15663    500001.0
20484    218600.0
9814     278000.0
           ...   
15362    263300.0
16623    266800.0
18086    500001.0
2144      72300.0
3665     151500.0
Name: median_house_value, Length: 4128, dtype: float64

In [31]:
X_test_prepared=full_pipeline.transform(X_test)
X_test_prepared

array([[ 0.28534728,  0.1951    , -0.28632369, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.06097472, -0.23549054,  0.11043502, ...,  0.        ,
         0.        ,  0.        ],
       [-1.42487026,  1.00947776,  1.85617335, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.23041404,  0.78014149, -0.28632369, ...,  0.        ,
         0.        ,  0.        ],
       [-0.08860699,  0.52740357,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.60445493, -0.66608108, -0.92113763, ...,  0.        ,
         0.        ,  0.        ]])

In [32]:
test_predicted=LR_model.predict(X_test_prepared)
test_predicted

array([ 51809.44720865, 129199.97279161, 267570.93719604, ...,
       439209.09136568, 121261.48178239, 181937.84269135])

In [33]:
pd.DataFrame({'Predicted_labels':test_predicted, 'Real_labels':Y_label})

,Predicted_labels,Real_labels
20046,51809.447209,47700.0
3024,129199.972792,45800.0
15663,267570.937196,500001.0
20484,269875.382250,218600.0
9814,264471.255881,278000.0
...,...,...
15362,222586.142332,263300.0
16623,245524.751800,266800.0
18086,439209.091366,500001.0
2144,121261.481782,72300.0


**sklearn.metrics** dan **mean_absolute_error** ni import qilamiz

In [35]:
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(Y_label,test_predicted)
print("MAE -",mae)

MAE - 50549.97538101806


**RMSE** ni ham sklearn.metrics dan import qilamiz

In [36]:
from sklearn.metrics import mean_squared_error

mse=mean_squared_error(Y_label,test_predicted)
print("RMSE -",np.sqrt(mse))

RMSE - 69829.55973661701
